## Import Packages

In [2]:
# import packages
# Importing all necessary packages

import numpy as np
import pandas as pd
#from sklearn.decomposition import PCA
import collections
from sklearn.model_selection import train_test_split

# importing os module
import os
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import mutual_info_classif, SelectKBest

from sklearn.feature_selection import RFECV
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
from sklearn import svm
from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
from sklearn.feature_selection import SelectFromModel

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
#import shap
import shap
import random
import scipy.stats

import warnings
warnings.filterwarnings("ignore")

/home/kajal.panchal/kajal/kajal/tools/miniconda3/install_loc/envs/HNSCC/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Functions

#### 1. Split Function: Splits the data into train and test

x: input

y:output

In [3]:
def split(x, y, random_state): # x and y are raw
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=random_state)
    # data without imputation and target values
    return x_train, x_test, y_train, y_test # _0 retaines sample type information _1 are without sample type info

#### 2. Random Selection: randomly selects a set of n features

In [4]:
def random_selection(num, all_features, random_state):
    # num is the feature set size to be selected
    # all_features = total pool of features
    # random selection of columns
    random.seed(random_state)
    features = []
    features = random.sample(all_features, num)
    return features

3. Min-Max Scaling


In [5]:
def minmax_scaling(x_train, x_test):

    scaler = MinMaxScaler()
    scaler.fit(x_train)
    x_train = pd.DataFrame(scaler.transform(x_train), columns=x_train.columns, index=x_train.index)
    x_test = pd.DataFrame(scaler.transform(x_test), columns=x_test.columns, index=x_test.index)
    return x_train, x_test


In [6]:
def impute(x_train, x_test):
    imputer = KNNImputer(n_neighbors=5)
    imputer.fit(x_train)
    x_train = pd.DataFrame(imputer.transform(x_train), columns=x_train.columns, index=x_train.index)
    x_test = pd.DataFrame(imputer.transform(x_test), columns=x_test.columns, index=x_test.index)
    return x_train, x_test

Evaluation

In [7]:
def eval(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, pos_label = "Tumor")
    precision = precision_score(y_true,y_pred, pos_label = "Tumor")
    recall = recall_score(y_true, y_pred, pos_label = "Tumor")
    auc = roc_auc_score(y_true, y_pred)
    return acc, f1, precision, recall

### BOXplot for model performance for selected vs random geneset

In [8]:
def measure_boxplot(df_measure_imp, df_measure_random, filepath, title):
    df_imp_long = df_measure_imp.melt(var_name="Model", value_name="Accuracy")
    df_imp_long["Type"] = "Important"
    df_rand_long = df_measure_random.melt(var_name="Model", value_name="Accuracy")
    df_rand_long["Type"] = "Random"
    df_all = pd.concat([df_imp_long, df_rand_long], ignore_index= True)

    # calculating p-val
    p_cv_logr = float(scipy.stats.wilcoxon(df_measure_imp["LogR"], df_measure_random["LogR"],alternative = "greater")[1])
    #print(p_cv_logr)
    p_cv_RF = float(scipy.stats.wilcoxon(df_measure_imp["RF"], df_measure_random["RF"], alternative = "greater")[1])
    #print(p_cv_RF)
    p_cv_MLP = float(scipy.stats.wilcoxon(df_measure_imp["MLP"], df_measure_random["MLP"], alternative = "greater")[1])
    #print(p_cv_MLP)
    p_cv_SVC = float(scipy.stats.wilcoxon(df_measure_imp["SVC"], df_measure_random["SVC"], alternative = "greater")[1])
    #print(p_cv_SVC)
    pvals = {"LogR":p_cv_logr, "RF":p_cv_RF, "MLP":p_cv_MLP, "SVC": p_cv_SVC}
    plt.figure(figsize=(10,7))
    ax = sns.boxplot(x="Model", y="Accuracy", hue="Type", data=df_all, palette="Set2")

    # Add p-values above boxes
    for i, model in enumerate(df_all["Model"].unique()):
        y = df_all[df_all["Model"]==model]["Accuracy"].max() + 0.002  # position above box
        p = pvals[model]
        ax.text(i, y, f"p = {p:.3e}", ha="center", va="bottom", fontsize=10, color="black")

    plt.title(title)
    plt.ylabel("Accuracy")
    plt.legend(title="Gene Set")  
    plt.savefig(filepath,  dpi=400, bbox_inches='tight')


#### Classification task

In [9]:
def classification_task(x_train, x_test, y_train, y_test, models, model_names, random_state):
        
        # accuracy dictionary
        accuracy_cv = {} # iteration over seeds
        accuracy_pred = {} # iteration over seeds
      
        i = 0
        for est in models: 
                model_name = model_names[i]
                i = i + 1
                est = est.fit(x_train, y_train)

                #  Stratified Cross validation
                cv = StratifiedKFold(n_splits=5, shuffle=True, random_state = random_state)
                y_pred_CV = cross_val_predict(est, x_train, y_train, cv=cv)               
                report_CV = classification_report(y_train, y_pred_CV, target_names=["Normal", "Tumor"], output_dict=True)
                acc = report_CV["accuracy"]
                accuracy_cv[model_name] = acc


                # prediction
                y_pred = est.predict(x_test)

                report = classification_report(y_test, y_pred, target_names=["Normal", "Tumor"], output_dict=True)
                acc = report["accuracy"]
                accuracy_pred[model_name] = acc
                # returns directory for models
        return accuracy_cv, accuracy_pred

### Classification on final geneset

### Feature Selection:Mutual Information

In [10]:
def get_miscores(X, y, random_state):
    mi_scores = mutual_info_classif(X, y, random_state=random_state)
    return mi_scores
def get_migenes(x, y, k, random_state):
    mi_df  = pd.DataFrame(columns=["gene", "MI_score"])
    mi_df["gene"] = x.columns
    random.seed(random_state)
    seeds = random.sample(range(0, 1000), 3)
    for seed in seeds: # you can increase the list of seeds
        mi_scores= get_miscores(x, y, random_state=seed)
        mi_df[seed] = mi_scores
        # Mi_score column is mean of all seeds
    mi_df["MI_score"] = mi_df.iloc[:, 1:4].mean(axis=1)

    mi_df = mi_df.sort_values(by="MI_score", ascending=False)
        
    top_1000_df = mi_df.head(k)
    top_genes_mi = mi_df["gene"].tolist()[:k]
    
    # returns the list and dataframe
    return top_genes_mi, top_1000_df 


### Feature Selection: RF-SelectFromModel

In [11]:
def RFSFM(x_train, x_test,y_train, y_test, random_state):
    # Using RandomForestClassifier as the estimator
    rf = RandomForestClassifier(n_estimators=500, random_state=random_state)
    # Using SelectFromModel to select top 100 features
    selector = SelectFromModel(rf, max_features=100, threshold="median")
    selector.fit(x_train, y_train)
    selected_features = x_train.columns[selector.get_support()]
    x_train_rf = x_train[selected_features]
    x_test_rf = x_test[selected_features]
    return x_train_rf, x_test_rf, selected_features

### Feature Selection: SVMRFE

In [12]:
def svm_rfe(x_train, y_train, features, random_state): # returns dataframe of features 
    # min = min numbers of features to be selected
    
    # cross validation to be used in RFECV
    cv = StratifiedKFold(n_splits = 5, shuffle = True, random_state = random_state)
    
    # using SVC estimator to check Features importance using RFE 
    svc = svm.SVC(kernel = "linear", random_state=random_state)
    # RFECV optimizes the minimum set of features needed for cassification task
    rfecv_SVM = RFECV(svc, min_features_to_select = 20, cv=cv, scoring='f1_weighted',  n_jobs=-1)
    fit_rfecv_SVM = rfecv_SVM.fit(x_train, y_train)
    
    RFECV_SVM_Top_Feat_df = pd.DataFrame({'Features': features, 'Selected': fit_rfecv_SVM.support_,
                                  'Rank': fit_rfecv_SVM.ranking_})
    RFECV_SVM_Top_Feat_df = RFECV_SVM_Top_Feat_df.sort_values(by = 'Rank')
 
    return RFECV_SVM_Top_Feat_df
    
    
    
    

#### Ploting ROC

In [13]:
# function to plot ROC curve
def roc(model_list, x_test, y_test):
    disp = RocCurveDisplay.from_estimator(model_list[0], x_test, y_test, pos_label='Tumor', name ="LogR", lw = 5)
    RocCurveDisplay.from_estimator(model_list[1], x_test, y_test, pos_label='Tumor', ax=disp.ax_, name ="RF", lw = 5);
    RocCurveDisplay.from_estimator(model_list[2], x_test, y_test, pos_label='Tumor', ax=disp.ax_, name ="MLP", lw = 5);
    RocCurveDisplay.from_estimator(model_list[3], x_test, y_test, pos_label='Tumor', ax=disp.ax_, name ="SVC", lw = 5); 
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.rc("font", size = 20)
    plt.legend(fontsize = "16", frameon = False)
    plt.title("100 genes (ANOVA)")
    plt.savefig("00-1_ROC_Top_100_Cancer.png", dpi=400, bbox_inches='tight')
    plt.show()


## MAIN

In [13]:
# loading all the genes selected in SVMRFE step
genes_r = pd.read_csv("results/SO/R/00-1_SVMRFE_GENEFreq_R.csv", index_col=0)
genes_r = list(genes_r.head(20).index)
genes_m = pd.read_csv("results/SO/M/00-1_SVMRFE_GENEFreq.csv", index_col=0)
genes_m = list(genes_m.head(20).index)
genes_p = pd.read_csv("results/SO/P/00-1_SVMRFE_GENEFreq_P.csv", index_col=0)
genes_p = list(genes_p.head(20).index)
genes_pp = pd.read_csv("results/SO/PP/00-1_SVMRFE_GENEFreq_PP.csv", index_col=0)
genes_pp = list(genes_pp.head(20).index)
genes_MO = pd.read_csv("results/MO/00-1_SVMRFE_GENEFreq_MO.csv", index_col=0)
genes_MO = list(genes_MO.head(20).index)


In [14]:
# Remove "_" and anything before it from each elemets in genes_MO

genes_MOwo = [gene.split("_", 1)[-1] for gene in genes_MO]
print(genes_MOwo)

['HPX', 'SERPINA6', 'ETFDH', 'NOSTRIN', 'SERPINA1', 'SH3BGRL2', 'ORM2', 'SERPING1', 'SERPINA7', 'ASPA', 'IFI35', 'SERPINA3', 'LOC105373265', 'PID1', 'ADAM12', 'LYVE1', 'QARS', 'MMRN1', 'TRPM6', 'AGT']


In [15]:
def classification_task_modified(x_train, x_test, y_train, y_test, models, model_names, random_state, omics):
        
        # accuracy dictionary
        accuracy_cv = {} # iteration over seeds
        accuracy_pred = {} # iteration over seeds

        shap_values = {}
        i = 0
        for est in models: 
                model_name = model_names[i]
                i = i + 1
                est = est.fit(x_train, y_train)

                #  Stratified Cross validation
                cv = StratifiedKFold(n_splits=5, shuffle=True, random_state = random_state)
                y_pred_CV = cross_val_predict(est, x_train, y_train, cv=cv)               
                report_CV = classification_report(y_train, y_pred_CV, target_names=["Normal", "Tumor"], output_dict=True)
                acc = report_CV["accuracy"]
                accuracy_cv[model_name] = acc


                # prediction
                y_pred = est.predict(x_test)

                report = classification_report(y_test, y_pred, target_names=["Normal", "Tumor"], output_dict=True)
                acc = report["accuracy"]
                accuracy_pred[model_name] = acc
                
                
        return accuracy_cv, accuracy_pred

In [70]:
# models to be used
def final_classification(x,y,features, omics):
    print("Starting final classification for", omics)
    print ("--------------------------------")

    x = x[features]

    dict_acc_cv = {"LogR":[], "RF":[], "MLP":[], "SVC":[]}
    dict_acc_pred = {"LogR":[], "RF":[], "MLP":[], "SVC":[]}
   
    for i in range(10):
        print("Iteration:", i+1)
        random_state = i
        # split the data
        x_train, x_test, y_train, y_test = split(x, y, random_state)
        
        
        if omics != "Transcriptomics":
            # imputation
            x_train, x_test = impute(x_train, x_test)

        # scaling
        x_train, x_test = minmax_scaling(x_train, x_test)
    

        
        logr = LogisticRegression(random_state=random_state, max_iter=800, solver='liblinear')
        rf = RandomForestClassifier(random_state=random_state, n_estimators=500)
        mlp = MLPClassifier(random_state=random_state, max_iter=800, activation="relu", solver='lbfgs', alpha=1e-5)
        svc = SVC(random_state=random_state, kernel = "linear", C = 0.1)

        models = [logr, rf, mlp, svc]
        model_names = ["LogR", "RF", "MLP", "SVC"]
    
        acc_cv, acc_pred = classification_task_modified(x_train, x_test, y_train, y_test, models, model_names, random_state, omics)    
        for key in acc_cv.keys():
            dict_acc_cv[key].append(acc_cv[key])
        for key in acc_pred.keys():
            dict_acc_pred[key].append(acc_pred[key])
            print(f"Accuracy for model {key}: {acc_pred[key]}")
        

        df_acc_cv = pd.DataFrame(dict_acc_cv)
        df_acc_cv.to_csv(f"results/top20_accuracy_cv_{omics}.csv")

       
        df_acc_pred = pd.DataFrame(dict_acc_pred)
        df_acc_pred.to_csv(f"results/top20_accuracy_pred_{omics}.csv")

       

    return df_acc_cv, df_acc_pred

        



In [16]:
data_r = pd.read_csv("data/processed/RNAseq_processed.csv", index_col=0)
data_m = pd.read_csv("data/processed/Methylation_processed.csv", index_col=0)
data_p = pd.read_csv("data/processed/Proteomics_processed.csv", index_col=0)
data_pp = pd.read_csv("data/processed/Phosphoproteomics_processed.csv", index_col=0)
data_mo = pd.read_csv("data/processed/All_processed.csv", index_col=0)
x_r = data_r.drop(columns=["Sample_Type"])
y_r = data_r["Sample_Type"]
x_m = data_m.drop(columns=["Sample_Type"])
y_m = data_m["Sample_Type"]
x_p = data_p.drop(columns=["Sample_Type"])
y_p = data_p["Sample_Type"]
x_pp = data_pp.drop(columns=["Sample_Type"])
y_pp = data_pp["Sample_Type"]
x_mo = data_mo.drop(columns=["PP_Sample_Type"])
y_mo = data_mo["PP_Sample_Type"]

# df_r_acc_cv, df_r_acc_pred = final_classification(x_r, y_r, genes_r, "Transcriptomics")
# df_m_acc_cv, df_m_acc_pred= final_classification(x_m, y_m, genes_m, "Methylation")
# df_p_acc_cv, df_p_acc_pred = final_classification(x_p, y_p, genes_p, "Proteomics")
# df_pp_acc_cv, df_pp_acc_pred= final_classification(x_pp, y_pp, genes_pp, "Phosphoproteomics")
# df_mo_acc_cv, df_mo_acc_pred = final_classification(x_mo, y_mo, genes_MO, "Multiomics")

In [17]:
data_r.shape, data_m.shape, data_p.shape, data_pp.shape, data_mo.shape

((162, 32298), (144, 3728), (172, 9659), (172, 6342), (131, 52024))

In [83]:
# make a dataframe of all accuracies cv column = omics types
#values will be mean +- std dev
df_all = pd.DataFrame()
df_all["Transcriptomics"] = df_r_acc_pred.mean().round(3).astype(str) + " ± " + df_r_acc_pred.std().round(3).astype(str)
df_all["Methylation"] = df_m_acc_pred.mean().round(3).astype(str) + " ± " + df_m_acc_pred.std().round(3).astype(str)
df_all["Proteomics"] = df_p_acc_pred.mean().round(3).astype(str) + " ± " + df_p_acc_pred.std().round(3).astype(str)
df_all["Phosphoproteomics"] = df_pp_acc_pred.mean().round(3).astype(str) + " ± " + df_pp_acc_pred.std().round(3).astype(str)
df_all["Multiomics"] = df_mo_acc_pred.mean().round(3).astype(str) + " ± " + df_mo_acc_pred.std().round(3).astype(str)
df_all.to_csv("results/Final_Accuracies_top20_all_omics.csv")


In [84]:
df_all

,Transcriptomics,Methylation,Proteomics,Phosphoproteomics,Multiomics
LogR,0.997 ± 0.01,0.993 ± 0.015,0.991 ± 0.014,1.0 ± 0.0,1.0 ± 0.0
RF,0.997 ± 0.01,0.997 ± 0.011,1.0 ± 0.0,0.991 ± 0.014,1.0 ± 0.0
MLP,0.997 ± 0.01,0.952 ± 0.044,0.991 ± 0.014,1.0 ± 0.0,1.0 ± 0.0
SVC,0.997 ± 0.01,0.983 ± 0.018,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0
